In [80]:
import pandas as pd
mc_data = pd.read_csv("piracydataset.csv", sep=',')
movie_data = pd.read_csv("movies_dataset.csv", sep=',')


#Minecraft Piracy Dataset
#Drop residual column
mc_data.drop(columns=['Unnamed: 0'])
#Because the individual usernames are not relevant to the analysis, drop them
#mc_data.drop(columns=['discordid', 'minecraftid']) 

#Movie Piracy Database
movie_data = movie_data.drop(columns=['Unnamed: 0','id'])
movie_data = movie_data.drop(columns=['storyline', 'writer', 'director'])
movie_data = movie_data.drop(columns=['run_time'])

ratings = movie_data['IMDb-rating']
na_counts = movie_data.isna().sum()
industries = movie_data['industry'].unique()

movies_by_industry = movie_data.groupby('industry')

Hollywood_movies = None

na_count = rating.isna().sum()

movie_data = movie_data.dropna()

movies_by_industry.first()

movies_by_industry.loc[(movies_by_industry['3D Movies'])]


AttributeError: 'DataFrameGroupBy' object has no attribute 'loc'